<a href="https://colab.research.google.com/github/masaki-cucu/Kaggle/blob/master/MNIST_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training, datasets, iterators, optimizers
from chainer.training import extensions
import numpy as np
from PIL import Image

batch_size = 10                # バッチサイズ10
uses_device = 0                # GPU#0を使用

In [0]:
class NMIST_Conv_NN(chainer.Chain):

	def __init__(self):
		super(NMIST_Conv_NN, self).__init__()
		with self.init_scope():
			self.conv1 = L.Convolution2D(1, 8, ksize=3)	# フィルタサイズ＝3で出力数8
			self.linear1 = L.Linear(1352, 10)			# 出力数10

	def __call__(self, x, t=None, train=True):
		# 畳み込みニューラルネットワークによる画像認識
		h1 = self.conv1(x)				# 畳み込み層
		h2 = F.relu(h1)					# 活性化関数
		h3 = F.max_pooling_2d(h2, 2)# プーリング層
		h4 = self.linear1(h3)			# 全結合層
		# 損失か結果を返す
		return F.softmax_cross_entropy(h4, t) if train else F.softmax(h4)

In [0]:
# ニューラルネットワークを作成
model = NMIST_Conv_NN()

# MNISTデータセットを用意する
train, test = chainer.datasets.get_mnist(ndim=3)

# 繰り返し条件を作成する
train_iter = iterators.SerialIterator(train, batch_size, shuffle=True)
test_iter = iterators.SerialIterator(test, batch_size, repeat=False, shuffle=False)

# 誤差逆伝播法アルゴリズムを選択する
optimizer = optimizers.Adam()
optimizer.setup(model)

# デバイスを選択してTrainerを作成する
updater = training.StandardUpdater(train_iter, optimizer)
trainer = training.Trainer(updater, (5, 'epoch'), out="result")
# テストをTrainerに設定する
trainer.extend(extensions.Evaluator(test_iter, model))
# 学習の進展を表示するようにする
#trainer.extend(extensions.ProgressBar())

# 機械学習を実行する
trainer.run()

# 学習結果を保存する
chainer.serializers.save_hdf5( 'chapt02.hdf5', model )

In [41]:
# 学習結果を読み込む
chainer.serializers.load_hdf5( 'chapt02.hdf5', model )

# 画像を読み込む
image = Image.open('../mnist-0.png').convert('L')
# ニューラルネットワークの入力に合わせて成形する
pixels = np.asarray(image).astype(np.float32).reshape(1,1,28,28)
pixels = pixels / 255

# ニューラルネットワークを実行する
result = model(pixels, train=False)
# 実行結果を表示する
for i in range(len(result.data[0])):
	print( str(i) + '\t' + str(result.data[0][i]) )


0	0.9999981
1	3.9182434e-17
2	1.1789126e-06
3	1.9131169e-10
4	2.0366011e-12
5	2.8351962e-10
6	5.680893e-07
7	2.2295162e-11
8	6.9278954e-08
9	9.433626e-10
